In [ ]:
import utils
import utils_preprocess
import utils_spatial

import numpy as np
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt

# STEP 1: Preprocessing PDSI and GLDAS data into Tabular Format

In [ ]:
# Process the pdsi netcdf files to obtain tabular data pickle file
pdsi_source_directory = r'C:\Users\saulg\Desktop\Remote_Data\pdsi'
pdsi_target_directory = r'C:\Users\saulg\Desktop\Remote_Data\pdsi_tabular'

utils_preprocess.process_pdsi_data(source_directory=pdsi_source_directory, target_directory=pdsi_target_directory)

In [ ]:
# Process the gldas netcdf files to obtain tabular data pickle file
gldas_source_directory = r'C:\Users\saulg\Desktop\Remote_Data\GLDAS'
gldas_target_directory = r'C:\Users\saulg\Desktop\Remote_Data\gldas_tabular'

utils_preprocess.process_gldas_data(source_directory=gldas_source_directory, target_directory=gldas_target_directory)

# Step 2: Transform PDSI, GLDAS, and Well Observations into format for ML

In [ ]:
# Load the shapefile
path_shape = '/home/saul/workspace/Well_Imputation/Aquifer Shapes/Beryl_Enterprise.shp'
aquifer_shape = utils.load_shapefile(path_shape)

In [ ]:
# Parse pdsi data and save it
directory_pdsi = r"/mnt/c/Users/saulg/Desktop/Remote_Data/pdsi_tabular"
pdsi = utils.pull_relevant_data(aquifer_shape, dataset_name="PDSI", dataset_directory=directory_pdsi)
utils.save_pickle(pdsi, "pdsi_data.pickle", "Datasets/")

In [ ]:
# Parse the GLDAS data and save it
directory_gldas = r"/mnt/c/Users/saulg/Desktop/Remote_Data/gldas_tabular"
gldas = utils.pull_relevant_data(aquifer_shape, dataset_name="GLDAS", dataset_directory=directory_gldas)
utils.save_pickle(gldas, "gldas_data.pickle", "Datasets/")

In [ ]:
# Process well data from csv files
well_locations = pd.read_csv("Aquifers Data/EscalanteBerylLocation.csv")
well_timeseries = pd.read_csv("Aquifers Data/EscalanteBerylTimeseries.csv")
data = utils.transform_well_data(well_timeseries, well_locations)
utils.save_pickle(data, "BerylEnterpriseData.pickle", "./Datasets/")


In [ ]:
# Plot the timeseries data to see if it looks reasonable
plt.plot(data["timeseries"], '-.')
plt.show()

# Step 3: Develop initial imputation model

# Step 4: Develop iterative refinement model

# Step 5: Analyze spatial characteristics of imputation model

In [ ]:
data = utils.load_pickle("Datasets/Well_Data_Imputed_iteration_2.pickle")

utils_spatial.krigging_interpolation(data = data,)

# Step 6: Calculate Storage Change

In [ ]:
raster = nc.Dataset("Datasets/well_data_iter_2.nc", 'r')
out = utils_spatial.StorageChangeCalculator()
ts = out.calulate_storage_curve(raster, date_range_filter=("1948-01-01", "2020-12-31"))